<a href="https://colab.research.google.com/github/yuja28/sw_Bootcamp/blob/main/ngrok.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
pip install fastapi

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.7/65.7 kB 1.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.0/67.0 kB 3.2 MB/s eta 0:00:00


In [4]:
pip install uvicorn

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.5/59.5 kB 1.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 3.7 MB/s eta 0:00:00


In [5]:
pip install ngrok

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 25.7 MB/s eta 0:00:00


In [6]:
pip install python-multipart

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.7/45.7 kB 1.1 MB/s eta 0:00:00


In [7]:
pip install pyngrok

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 681.2/681.2 kB 8.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyngrok: filename=pyngrok-6.0.0-py3-none-any.whl size=19867 sha256=299cb4eb43ae2714eef5b9e23b8015f1a9e73c38c1fde5252a958da63526cb5f
  Stored in directory: /root/.cache/pip/wheels/5c/42/78/0c3d438d7f5730451a25f7ac6cbf4391759d22a67576ed7c2c
Successfully built pyngrok


In [8]:
!ngrok authtoken 2Ss4L0fCzwPofSVGgayOeSrawxl_6Mf5Hzf5EqzKaLqCxoH81

Authtoken saved to configuration file: /root/.ngrok2/ngrok.yml


In [9]:
from typing import Union
from fastapi import FastAPI, UploadFile, File
from tensorflow.keras.models import load_model
from starlette.requests import Request
from fastapi import FastAPI, HTTPException, UploadFile, File
from starlette.requests import Request
from starlette.responses import HTMLResponse
from tensorflow.keras.models import load_model
from PIL import Image
import numpy as np
import io

app = FastAPI()

# 모델 불러오기
model = load_model('/content/drive/MyDrive/mvtecv/bottle.h5')

@app.post("/predict")
async def predict(file: UploadFile = File(...)):
    # 이미지 불러오기 및 전처리
    image = Image.open(io.BytesIO(await file.read()))
    image = image.resize((28, 28)).convert('L')
    image = np.array(image) / 255.0
    image = image.reshape(-1, 28, 28, 1)

    # 예측
    pred = model.predict(image)
    result = np.argmax(pred)

    return {"result": int(result)}

@app.get("/", response_class=HTMLResponse)
async def read_root(request: Request):
    return """
    <html>
        <body>
            <form action="/predict" enctype="multipart/form-data" method="post">
                <input name="file" type="file">
                <input type="submit">
            </form>
        </body>
    </html>
    """

@app.get("/items/{item_id}")
def read_item(item_id: int, q: str = None):
    return {"item_id": item_id, "q": q}

In [13]:
from pyngrok import ngrok
import uvicorn
import threading

url = ngrok.connect(8000)  # FastAPI의 기본 포트는 8000입니다.
print('Public URL:', url)

def run():
    uvicorn.run(app, host="0.0.0.0", port=8000)

threading.Thread(target=run).start()

Public URL: NgrokTunnel: "https://50b4-104-154-208-44.ngrok-free.app" -> "http://localhost:8000"


INFO:     Started server process [388]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://0.0.0.0:8000 (Press CTRL+C to quit)


In [ ]:
!lsof -i :8000